## Example for ImArray
ImArray is a software to image an object in the sky by an array of ground Telescopes
Copyright (C) 2016  Thomas Vuillaume

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>

The author may be contacted @
thomas.vuillaume@lapp.in2p3.fr

Let's start with some imports

In [17]:
import geometry as geo
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, PathPatch
import time
import math
import object as obj
import CameraImage as ci
import Hillas as hillas
import os
import vizualisation as viz

We can define some shower parameters:

In [18]:
impact_point = np.array([0,250,0])

#shower direction
salt = math.radians(74)
saz = math.radians(0)

#pointing direction
talt = math.radians(74)
taz = math.radians(0)

#shower parameters
shower_top = 15000
shower_length = 12000
shower_width = 200

npoints = 500

Here, our shower is a geometrical object (an ellpisoid in this case) randomly filled with points:

In [20]:
shower = obj.random_ellipsoide(shower_top, shower_length, shower_width, salt, saz, impact_point, npoints)

Let's now define a telescopes array configuration:

In [21]:
tel_normal = geo.altaz_to_normal(talt,taz)

tel1 = geo.Telescope([500,100,-100],tel_normal,0)
tel2 = geo.Telescope([-10,300,100],tel_normal,0)
tel3 = geo.Telescope([-500,400,10],tel_normal,0)
tel4 = geo.Telescope([-350,-500,0],tel_normal,0)
tel5 = geo.Telescope([600,-100,150],tel_normal,0)
tel6 = geo.Telescope([300,-300,50],tel_normal,0)
tel7 = geo.Telescope([80,0,0],tel_normal,0)
tel8 = geo.Telescope([-80,0,0],tel_normal,0)
tel9 = geo.Telescope([0,-80,0],tel_normal,0)
tel10 = geo.Telescope([0,80,0],tel_normal,0)

alltel = [tel1, tel2, tel3, tel4, tel5, tel6]

The telecopes positions can also be load from a data file with:

In [5]:
#alltel = geo.load_telescopes("data/tel_pos.dat")

We can check that no pair of telescopes are set at the same location:

In [6]:
for tel1 in alltel:
    for tel2 in alltel:
        if ((tel1.center == tel2.center).all() and tel1.id != tel2.id):
            print(tel1.id, tel2.id, tel1.center, tel2.center)

Let's vizualise what we have done so far:

In [22]:
viz.plot_shower3d(shower,alltel)

In [8]:
IM = []
visible_points = []
HillasParameters = []
noise = 1
allhist = np.zeros(1855) ###THIS NEEDS TO BE CHANGED

In [23]:
for tel in alltel:
    X = []
    Y = []
    for point in shower:
        im = geo.image_point_pfo(point, tel)
        IM.append(im)
        center_camera = geo.camera_center(tel)
        if geo.is_point_in_camera_plane(im, tel):
            visible_points.append(geo.is_point_visible(im,tel))
        im_cam = geo.site_to_camera_cartesian([im[0],im[1],im[2]], tel)
        X.append(im_cam[0])
        Y.append(im_cam[1])
    #the camera histogram corresponds to the image : a list of pixels positions + signal 
    hist = ci.camera_image(tel, np.column_stack((X,Y)), "results/{}.txt".format(tel.id), noise)
    hp = hillas.hillas_parameters_2(hist[:,0], hist[:,1], hist[:,2])
    hp.append(tel.id)

    HillasParameters.append(hp)
    allhist += hist[:,2]
    
    #for each telescope, we can plot the camera image:
    plt.plot(X,Y,'o', label = tel.center,markersize=3)

One can vizualise the camera images stacked together:

In [24]:
plt.legend(loc='upper center', fancybox=True, ncol=3, bbox_to_anchor=(0.5,1.1))
plt.show()

We can now make two Hillas reconstruction:

In [25]:
pa = hillas.impact_parameter_average(alltel, HillasParameters, talt, taz)
p = hillas.impact_parameter_ponderated(alltel, HillasParameters, talt, taz)

And compare the results:

In [26]:
print("Real impact parameter : ", impact_point)
print("Reconstruction with simple average = %s \tError = %.2fm" % (pa, math.sqrt(((impact_point-pa)**2).sum())))
print("Reconstruction with ponderation and cut = %s \tError = %.2fm" % (p, math.sqrt(((impact_point-p)**2).sum())))

Real impact parameter :  [  0 250   0]
Reconstruction with simple average = [-174.7190883705895, 310.5207559013316, 0.0] 	Error = 184.90m
Reconstruction with ponderation and cut = [-108.71020792616473, 303.52985906870407, 0.0] 	Error = 121.17m
